# Data Wrangling
    - For bulid neural network to predict the blood glucose levels, the data used in this project provided by Michael Kahn, MD, PhD, Washington University, St. Louis, MO. 
    
    Data Set Information:

    Diabetes patient records were obtained from two sources: an automatic electronic recording device and paper records. The automatic device had an internal clock to timestamp events, whereas the paper records only provided "logical time" slots (breakfast, lunch, dinner, bedtime). For paper records, fixed times were assigned to breakfast (08:00), lunch (12:00), dinner (18:00), and bedtime (22:00). Thus paper records have fictitious uniform recording times whereas electronic records have more realistic time stamps.

Diabetes files consist of four fields per record. Each field is separated by a tab and each record is separated by a newline.

    File Names and format:
    (1) Date in MM-DD-YYYY format
    (2) Time in XX:YY format
    (3) Code
    (4) Value

File Names and format:

        (1) Date in MM-DD-YYYY format
        (2) Time in XX:YY format
        (3) Code
        (4) Value

The Code field is deciphered as follows:

    33 = Regular insulin dose
    34 = NPH insulin dose
    35 = UltraLente insulin dose
    48 = Unspecified blood glucose measurement
    57 = Unspecified blood glucose measurement
    58 = Pre-breakfast blood glucose measurement
    59 = Post-breakfast blood glucose measurement
    60 = Pre-lunch blood glucose measurement
    61 = Post-lunch blood glucose measurement
    62 = Pre-supper blood glucose measurement
    
The aim:

      to prepare the data to be able to use as neural network inputs.

In [391]:
import pandas as pd
import numpy as np

In [392]:
col = ["date","time","code","value"]
df = pd.read_csv("data-29",sep='\t',names=col,)

In [393]:
df.head()

,date,time,code,value
0,04-29-1990,7:00,58,136
1,04-29-1990,7:00,33,007
2,04-29-1990,7:00,34,018
3,04-29-1990,14:00,60,062
4,04-29-1990,14:00,33,004


In [394]:
df.pop("time")

0        7:00
1        7:00
2        7:00
3       14:00
4       14:00
        ...  
1284    15:00
1285    15:00
1286    18:00
1287    18:00
1288    22:00
Name: time, Length: 1289, dtype: object

In [395]:
df = pd.pivot_table(df, values='value', index=["date"],columns=['code'], aggfunc=np.sum,dropna=True,fill_value=0)

In [396]:
df.head()

code,0,33,34,48,58,60,62
date,,,,,,,
04-29-1990,0,007004006,018,102,136,062,081
04-30-1990,0,007004006,018,162,097,132,144
05-01-1990,0,007004006,018,180,183,192,139
05-02-1990,0,007004006,018,162,151,102,135
05-03-1990,0,007004006,018,150,167,066,097


In [397]:
# make string version of original column, call it 'col'
df['col'] = df[33].astype(str)

# make the new columns using string indexing
df['33_1'] = df['col'].str[2:3]
df['33_2'] = df['col'].str[5:6]
df['33_3'] = df['col'].str[8:9]

In [398]:
remove_col = [0,33,"col"]
for i in remove_col : df.pop(i)

In [399]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159 entries, 04-29-1990 to 12-16-1990
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   34      159 non-null    object
 1   48      159 non-null    object
 2   58      159 non-null    object
 3   60      159 non-null    object
 4   62      159 non-null    object
 5   33_1    159 non-null    object
 6   33_2    159 non-null    object
 7   33_3    159 non-null    object
dtypes: object(8)
memory usage: 11.2+ KB


In [390]:
df.head()

code,34,48,58,60,62,65,33_1,33_2,33_3
date,,,,,,,,,
04-21-1991,13,123,100,0,119,0,,,
04-22-1991,13,0,216,0,211,0,,,
04-23-1991,13,0,257,0,129,0,,,
04-24-1991,14,340,239,0,129,0,,,
04-25-1991,14,288,67,0,206,0,,,


In [352]:
df.to_csv ("combined.csv")

Take a look with Excel

In [353]:
#the another aim of save and read tha data again to convert the types of columns.
df = pd.read_csv("combined.csv",index_col="date")
df.head()

,34,48,58,60,62,33_1,33_2,33_3
date,,,,,,,,
04-29-1990,18,102,136,62,81,7,4.0,6.0
04-30-1990,18,162,97,132,144,7,4.0,6.0
05-01-1990,18,180,183,192,139,7,4.0,6.0
05-02-1990,18,162,151,102,135,7,4.0,6.0
05-03-1990,18,150,167,66,97,7,4.0,6.0


In [354]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159 entries, 04-29-1990 to 12-16-1990
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   34      159 non-null    int64  
 1   48      159 non-null    int64  
 2   58      159 non-null    int64  
 3   60      159 non-null    int64  
 4   62      159 non-null    int64  
 5   33_1    159 non-null    int64  
 6   33_2    158 non-null    float64
 7   33_3    148 non-null    float64
dtypes: float64(2), int64(6)
memory usage: 11.2+ KB


In [355]:
df.isnull().sum()

34       0
48       0
58       0
60       0
62       0
33_1     0
33_2     1
33_3    11
dtype: int64

    After take a look at the dataframe after saving: I found less data in some columns.
    but in columns like 34,33_1,33_2 and 33_3 I cant use mean to fill the empty cells becuase that a fixed number.
        - I will fill cells manually.
    but in 48,58,60 and 62 I will use the mean to fill the empty cells on these columns.
        - I use pandas mask to fill with mean 

In [356]:
df.to_csv ("manual.csv")

In [357]:
df = pd.read_csv("manual.csv",index_col="date")
df.head()

,34,48,58,60,62,33_1,33_2,33_3
date,,,,,,,,
4/29/1990,18,102,136,62,81,7,4,6
4/30/1990,18,162,97,132,144,7,4,6
5/1/1990,18,180,183,192,139,7,4,6
5/2/1990,18,162,151,102,135,7,4,6
5/3/1990,18,150,167,66,97,7,4,6


In [358]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159 entries, 4/29/1990 to 12/16/1990
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   34      159 non-null    int64
 1   48      159 non-null    int64
 2   58      159 non-null    int64
 3   60      159 non-null    int64
 4   62      159 non-null    int64
 5   33_1    159 non-null    int64
 6   33_2    159 non-null    int64
 7   33_3    159 non-null    int64
dtypes: int64(8)
memory usage: 11.2+ KB


In [359]:
#fill the remaning columns by mean
df=df.mask(df==0).fillna(df.mean())

In [360]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159 entries, 4/29/1990 to 12/16/1990
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   34      159 non-null    int64  
 1   48      159 non-null    float64
 2   58      159 non-null    float64
 3   60      159 non-null    float64
 4   62      159 non-null    int64  
 5   33_1    159 non-null    int64  
 6   33_2    159 non-null    float64
 7   33_3    159 non-null    int64  
dtypes: float64(4), int64(4)
memory usage: 11.2+ KB


In [361]:
#save and take a look again
df.to_csv ("manual2.csv")

In [362]:
df = pd.read_csv("manual2.csv",index_col="date")
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159 entries, 4/29/1990 to 12/16/1990
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   34      159 non-null    int64  
 1   48      159 non-null    float64
 2   58      159 non-null    float64
 3   60      159 non-null    float64
 4   62      159 non-null    int64  
 5   33_1    159 non-null    int64  
 6   33_2    159 non-null    int64  
 7   33_3    159 non-null    int64  
dtypes: float64(3), int64(5)
memory usage: 11.2+ KB


    -change one cell in 33-2

### cleaning code

In [363]:
df.isnull().sum()

34      0
48      0
58      0
60      0
62      0
33_1    0
33_2    0
33_3    0
dtype: int64

In [364]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159 entries, 4/29/1990 to 12/16/1990
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   34      159 non-null    int64  
 1   48      159 non-null    float64
 2   58      159 non-null    float64
 3   60      159 non-null    float64
 4   62      159 non-null    int64  
 5   33_1    159 non-null    int64  
 6   33_2    159 non-null    int64  
 7   33_3    159 non-null    int64  
dtypes: float64(3), int64(5)
memory usage: 11.2+ KB


In [365]:
#convert 48,58,60 columns from float to int
df = df.astype({"48":int})
df = df.astype({"58":int})
df = df.astype({"60":int})

In [366]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159 entries, 4/29/1990 to 12/16/1990
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   34      159 non-null    int64
 1   48      159 non-null    int32
 2   58      159 non-null    int32
 3   60      159 non-null    int32
 4   62      159 non-null    int64
 5   33_1    159 non-null    int64
 6   33_2    159 non-null    int64
 7   33_3    159 non-null    int64
dtypes: int32(3), int64(5)
memory usage: 9.3+ KB


    Change the name of columns to data name
    33 = Regular insulin dose
    34 = NPH insulin dose
    48 = Unspecified blood glucose measurement
    58 = Pre-breakfast blood glucose measurement
    60 = Pre-lunch blood glucose measurement
    62 = Pre-supper blood glucose measurement

In [367]:
#move Regular insulin dose to be in first
df = df[["33_1","33_2","33_3","34","48","58","60","62"]]

In [368]:
#Change the name of columns to be more understandable
df.rename(columns={"34": "NPH insulin",
                   "48":"Unspecified_BGM",
                   "58":"Pre_breakfast_BGM",
                   "60":"Pre_lunch_BGM",
                   "62":"Pre-supper_BGM",
                   "33_1":"Regular_insulin_D1",
                   "33_2":"Regular_insulin_D2",
                   "33_3":"Regular_insulin_D3"},inplace=True)

In [369]:
df

,Regular_insulin_D1,Regular_insulin_D2,Regular_insulin_D3,NPH insulin,Unspecified_BGM,Pre_breakfast_BGM,Pre_lunch_BGM,Pre-supper_BGM
date,,,,,,,,
4/29/1990,7,4,6,18,102,136,62,81
4/30/1990,7,4,6,18,162,97,132,144
5/1/1990,7,4,6,18,180,183,192,139
5/2/1990,7,4,6,18,162,151,102,135
5/3/1990,7,4,6,18,150,167,66,97
...,...,...,...,...,...,...,...,...
12/12/1990,6,5,6,20,174,227,198,181
12/13/1990,6,5,6,20,160,225,169,99
12/14/1990,6,5,6,20,143,234,196,122


In [370]:
df.to_csv("final.csv")